In [1]:
import tensorflow as tf
from tensorflow.keras import *
import keras.backend as K
import copy
import os, sys
os.chdir('..')
join = os.path.join(os.getcwd(), '_global')
sys.path.extend([join])
from _global.config import *
from _global.funcs import *

In [2]:
LEARNING_RATES = [1e-4, 1e-3, 1e-2, 1e-1]
OPTIMIZIERS = [optimizers.RMSprop(), optimizers.Adadelta(), optimizers.Adagrad(), optimizers.Adamax(),
               optimizers.Ftrl(), optimizers.Nadam(), optimizers.SGD()]
LOSS_FNS = [losses.MeanAbsoluteError(), losses.MeanSquaredError(), losses.MeanSquaredLogarithmicError()]
ONE_EPOCH_THRESHOLD = 70
TWO_EPOCH_THRESHOLD = 55
THREE_EPOCH_THRESHOLD = 30
FOUR_EPOCH_THRESHOLD = 25
THESHOLDS = [70, 55, 30, 25]

train_filenames = [OUTPUT_TRAIN_TFRECORD]
train_dataset = prepare_dataset(train_filenames)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [3]:
raw_model = models.load_model(RAW_MODEL)

In [4]:
OPTIMIZERS_WITH_LR = []
for optimizer in OPTIMIZIERS:
    for lr in LEARNING_RATES:
        temp = copy.deepcopy(optimizer)
        K.set_value(temp.learning_rate, lr)
        OPTIMIZERS_WITH_LR.append(temp)

In [5]:
def is_loss_under_threshold(model, epoch):
    losses = model.history.history['loss']
    return losses[len(losses)-1] < THESHOLDS[epoch-1]


def fit_model(model, epochs=1):
    model.fit(train_dataset, epochs=epochs)
    if epochs > 4 or not is_loss_under_threshold(model, epochs):
        return
    fit_model(model, epochs+1)

In [6]:
def get_statistic(model, detailed=False):
    eval_acc = lambda x: x
    eval_loss = eval_acc
    if not detailed:
        eval_acc = lambda x: max(x)
        eval_loss = lambda x: min(x)

    hist = model.history.history
    return {
        'optimizer': model.optimizer.get_config()['name'],
        'learning_rate': model.optimizer.get_config()['learning_rate'],
        'loss_fn': model.loss.name,
        'accuracy': eval_acc(hist['accuracy']),
        'loss': eval_loss(hist['loss']),
    }


In [7]:
def statistic_item_tostring(stat):
    str = ""
    for key in stat:
        str += f"{key}: {stat[key]}, "
    return str + "\n"


In [8]:
len(OPTIMIZERS_WITH_LR)

28

In [9]:
len(OPTIMIZERS_WITH_LR)

28

In [10]:
statistics = []
output_path = os.path.join(INSTALLATION_PATH, "hyper_parameter_tuning.txt")
for optimizer_with_lr in OPTIMIZERS_WITH_LR:
    for loss_fn in LOSS_FNS:
        raw_model.compile(optimizer=optimizer_with_lr, loss=loss_fn, metrics='accuracy')
        fit_model(raw_model)
        stat = get_statistic(raw_model, detailed=True)
        statistics.append(stat)
        stat_str = statistic_item_tostring(stat)
        print(stat_str)
        content = ""
        if os.path.exists(output_path):
            content = tf.io.read_file(output_path)
        tf.io.write_file(output_path, content + stat_str)

InternalError:  Attempting to perform BLAS operation using StreamExecutor without BLAS support
	 [[node sequential/dense/Tensordot/MatMul (defined at C:\Users\niklas\AppData\Local\Temp/ipykernel_27332/3724473256.py:7) ]] [Op:__inference_train_function_10036]

Function call stack:
train_function


In [ ]:
statistics